##EX2

In [1]:
import csv
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
from itertools import cycle
from scipy import interp


df = pd.read_csv('1.data', delimiter = ",", names=['age', 'sex', 'cp', 'restbp', 'chol', 'fbs', 'restecg', 
           'thalach', 'exang', 'oldpeak', 'slope', 'ca','thal', 'num'], encoding = "ISO-8859-1")

print(df)

training_x=df.iloc[1:df.shape[0],0:13]

training_y=df.iloc[1:df.shape[0],13:14]

x=np.array(training_x)
y=np.array(training_y)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.25,random_state=None)

clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(X_train, Y_train)
prediction_prob = clf.predict_proba(X_test)
prediction = clf.predict(X_test)
print(prediction[:10])
accuracy = clf.score(X_test, Y_test)
print('The accuracy is: {}'.format(int(accuracy*1000)/10))
print(confusion_matrix(Y_test, prediction, labels=[0, 1, 2, 3, 4]))
report = classification_report(Y_test, prediction)
print(report)
roc_auc_score(Y_test, prediction_prob, multi_class='ovr')

      age  sex   cp  restbp   chol  fbs  restecg  thalach  exang  oldpeak  \
0    63.0  1.0  1.0   145.0  233.0  1.0      2.0    150.0    0.0      2.3   
1    67.0  1.0  4.0   160.0  286.0  0.0      2.0    108.0    1.0      1.5   
2    67.0  1.0  4.0   120.0  229.0  0.0      2.0    129.0    1.0      2.6   
3    37.0  1.0  3.0   130.0  250.0  0.0      0.0    187.0    0.0      3.5   
4    41.0  0.0  2.0   130.0  204.0  0.0      2.0    172.0    0.0      1.4   
..    ...  ...  ...     ...    ...  ...      ...      ...    ...      ...   
298  45.0  1.0  1.0   110.0  264.0  0.0      0.0    132.0    0.0      1.2   
299  68.0  1.0  4.0   144.0  193.0  1.0      0.0    141.0    0.0      3.4   
300  57.0  1.0  4.0   130.0  131.0  0.0      0.0    115.0    1.0      1.2   
301  57.0  0.0  2.0   130.0  236.0  0.0      2.0    174.0    0.0      0.0   
302  38.0  1.0  3.0   138.0  175.0  0.0      0.0    173.0    0.0      0.0   

     slope   ca  thal  num  
0      3.0  0.0   6.0    0  
1      2.0  3.0  

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7082738420999289

##EX3

In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
training_x=df.iloc[1:df.shape[0],0:13]

training_y=df.iloc[1:df.shape[0],13:14]

x=np.array(training_x)
y=np.array(training_y)
k = 5
k_fold = StratifiedKFold(n_splits=k, random_state=42, shuffle=True)
smoothing_factor_option = [1, 2, 3, 4, 5, 6]
fit_prior_option = [True, False]
auc_record = {}

for train_indices, test_indices in k_fold.split(x, y):
  X_train, X_test = x[train_indices], x[test_indices]
  Y_train, Y_test = y[train_indices], y[test_indices]
  for alpha in smoothing_factor_option:
    if alpha not in auc_record:
      auc_record[alpha] = {}
    for fit_prior in fit_prior_option:
      clf = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
      clf.fit(X_train, Y_train)
      prediction_prob = clf.predict_proba(X_test)
      pos_prob = prediction_prob
      auc = roc_auc_score(Y_test, pos_prob, multi_class='ovr')
      auc_record[alpha][fit_prior] = auc + auc_record[alpha].get(fit_prior, 0.0)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [3]:
for smoothing, smoothing_record in auc_record.items():
  for fit_prior, auc in smoothing_record.items():
    print(' {} {} {}'.format(smoothing, fit_prior,  auc/k))

 1 True 0.7387683082110068
 1 False 0.7240081197527347
 2 True 0.7409993111383713
 2 False 0.7282129501516407
 3 True 0.7429084252482335
 3 False 0.733468048862163
 4 True 0.7443530988131648
 4 False 0.7345989320375252
 5 True 0.744054021365943
 5 False 0.7378868982035149
 6 True 0.7449290918508632
 6 False 0.7398217647949907


In [4]:
clf = MultinomialNB(alpha=6.0, fit_prior=True)

clf.fit(X_train, Y_train)
pos_prob = clf.predict_proba(X_test)
print('AUC with the best model:', roc_auc_score(Y_test, pos_prob, multi_class='ovr'))

AUC with the best model: 0.7268021493075578


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
